# Linear Regression

**Linear Regression** is a simple machine learning model where the response y is modelled by a linear combination of the predictors in X.

The model can take array-like objects, either in host as NumPy arrays or in device (as Numba or cuda_array_interface-compliant), as well as cuDF DataFrames as the input. 

For information about cuDF, refer to the [cuDF documentation](https://docs.rapids.ai/api/cudf/stable).

For information about cuML's linear regression API: https://docs.rapids.ai/api/cuml/stable/api.html#cuml.LinearRegression

**NOTE:** This notebook is not expected to run on a GPU with under 16GB of RAM with its current value for `n_smaples`.  Please change `n_samples` from `2**20` to `2**19`

## Imports

In [1]:
import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

## Define Parameters

In [2]:
n_samples = 2**20 #If you are running on a GPU with less than 16GB RAM, please change to 2**19 or you could run out of memory
n_features = 399

random_state = 23

## Generate Data

In [3]:
%%time
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=random_state)

X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]

X_cudf, X_cudf_test, y_cudf, y_cudf_test = train_test_split(X, y, test_size = 0.2, random_state=random_state)

CPU times: user 872 ms, sys: 199 ms, total: 1.07 s
Wall time: 1.08 s


In [4]:
# Copy dataset from GPU memory to host memory.
# This is done to later compare CPU and GPU results.
X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

## Scikit-learn Model

### Fit, predict and evaluate

In [5]:
%%time
ols_sk = skLinearRegression(fit_intercept=True,
                            n_jobs=-1)

ols_sk.fit(X_train, y_train)

CPU times: user 1min 16s, sys: 55.1 s, total: 2min 11s
Wall time: 16.7 s


LinearRegression(n_jobs=-1)

In [6]:
%%time
predict_sk = ols_sk.predict(X_test)

CPU times: user 429 ms, sys: 1.05 s, total: 1.48 s
Wall time: 102 ms


In [7]:
%%time
r2_score_sk = r2_score(y_cudf_test, predict_sk)

CPU times: user 214 ms, sys: 791 ms, total: 1 s
Wall time: 34 ms


## cuML Model

### Fit, predict and evaluate

In [8]:
%%time
ols_cuml = cuLinearRegression(fit_intercept=True,
                              algorithm='eig')

ols_cuml.fit(X_cudf, y_cudf)

CPU times: user 470 ms, sys: 1.62 s, total: 2.09 s
Wall time: 156 ms


LinearRegression()

In [9]:
%%time
predict_cuml = ols_cuml.predict(X_cudf_test)

CPU times: user 44.5 ms, sys: 6.51 ms, total: 51.1 ms
Wall time: 50.5 ms


In [10]:
%%time
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)

CPU times: user 1.41 ms, sys: 1.09 ms, total: 2.5 ms
Wall time: 2.51 ms


## Compare Results

In [11]:
print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (cuML): %s" % r2_score_cuml)

R^2 score (SKL):  1.0
R^2 score (cuML): 1.0
